## 4. Context Engineering

<div style="text-align: right"> Initial issue : 2025.11.05 </div>
<div style="text-align: right"> last update : 2025.11.05 </div>

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

**에이전트가 실패하는 이유**

에이전트가 실패할 때는 일반적으로 에이전트 내부의 LLM 호출이 잘못된 작업을 수행하거나 예상대로 작동하지 않았기 때문입니다.    
LLM은 다음 두 가지 이유 중 하나로 실패합니다:

1. 기본 LLM이 충분히 능력이 없음
2. "올바른" 컨텍스트가 LLM에 전달되지 않음

대부분의 경우 실제로는 두 번째 이유가 에이전트의 신뢰성을 떨어뜨립니다.

**Context Engineering**은 LLM이 작업을 완수할 수 있도록 올바른 형식으로 올바른 정보와 도구를 제공하는 것입니다.   
이것이 AI 엔지니어의 가장 중요한 업무입니다.

### 컨텍스트의 종류


에이전트는 세 가지 종류의 컨텍스트를 제어합니다:

| 컨텍스트 타입 | 제어 대상 | 지속성 |
|------------|---------|-------|
| **Model Context** | 모델 호출에 들어가는 내용 (지시사항, 메시지 기록, 도구, 응답 형식) | Transient |
| **Tool Context** | 도구가 액세스하고 생성하는 내용 (상태, 저장소, 런타임 컨텍스트에 읽기/쓰기) | Persistent |
| **Life-cycle Context** | 모델 및 도구 호출 사이에 발생하는 작업 (요약, 가드레일, 로깅 등) | Persistent |

**Transient Context**: LLM이 단일 호출에서 보는 내용. 상태에 저장된 내용을 변경하지 않고 메시지, 도구 또는 프롬프트를 수정할 수 있습니다.

**Persistent Context**: 여러 턴에 걸쳐 상태에 저장되는 내용. 라이프사이클 훅과 도구 쓰기는 이를 영구적으로 수정합니다.

### 데이터 소스

에이전트는 다양한 데이터 소스에 액세스(읽기/쓰기)합니다:

| 데이터 소스 | 다른 이름 | 범위 | 예시 |
|----------|---------|------|-----|
| **Runtime Context** | 정적 구성 | 대화 범위 | 사용자 ID, API 키, DB 연결, 권한 |
| **State** | 단기 메모리 | 대화 범위 | 현재 메시지, 업로드된 파일, 인증 상태 |
| **Store** | 장기 메모리 | 대화 간 공유 | 사용자 선호도, 추출된 인사이트, 기록 데이터 |

## Model Context

각 모델 호출에 들어가는 내용을 제어합니다 - 지시사항, 사용 가능한 도구, 사용할 모델 및 출력 형식입니다.

### System Prompt

시스템 프롬프트는 LLM의 동작과 능력을 설정합니다. 다양한 사용자, 컨텍스트 또는 대화 단계에는 다양한 지시사항이 필요합니다.

### Messages  

메시지는 LLM에 전송되는 프롬프트를 구성합니다.  
LLM이 올바른 정보를 가지고 잘 응답할 수 있도록 메시지 내용을 관리하는 것이 중요합니다.

state에서 파일 컨텍스트 주입

In [5]:
from langchain.agents.middleware import wrap_model_call, ModelResponse, ModelRequest
from typing import Callable

@wrap_model_call # 모델 호출을 감싸는 미들웨어로 등록, 모델 호출 전후에 추가로직 실행
def inject_file_context(
    request: ModelRequest, handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """사용자가 업로드한 파일 컨텍스트를 주입"""
    # State에서 업로드된 파일 메타데이터 가져오기
    uploaded_files = request.state.get("uploaded_files", [])

    if uploaded_files:
        # 사용 가능한 파일에 대한 컨텍스트 구축
        file_descriptions = []
        for file in uploaded_files:
            file_descriptions.append(
                f"- {file['name']} ({file['type']}): {file['summary']}"
            ) # 각 파일을 약속된 형식, 이름으로 변환?? (예: - report.pdf (PDF): 2024년 분기별 매출 보고서)

        file_context = f"""Files you have access to in this conversation:
{chr(10).join(file_descriptions)}

Reference these files when answering questions.""" # 파일 컨텍시트 구성

        # 최근 메시지 앞에 파일 컨텍스트 주입
        messages = [
            *request.messages,
            {"role": "user", "content": file_context},
        ]
        request = request.override(messages=messages)
        # 기존 대화 메시지 뒤에 파일 정보를 user 메시지로 추가(LLM이 파일 존재를 인지하고 답변에 사용할 수 있게됨)

    return handler(request) # 모델 호출 실행


# 이렇게 하면 보고서 내용 요약해줘 질문에서 LLM은 어떤 보고서가 있는지 모르겠지만
# 미들웨어를 적용하면 LLM이 파일을 인지하고 적절한 도구를 호출하거나 맥락을 고려한 답변 가능

실행 예시

```python
agent = create_agent(model=model, tools=[search_tool], middleware=[inject_file_context])

# 파일이 업로드된 상태로 호출
result = agent.invoke(
    {
        "messages": [{"role": "user", "content": "What files do I have?"}],
        "uploaded_files": [
            {"name": "report.pdf", "type": "PDF", "summary": "Q4 sales report"},
            {"name": "data.csv", "type": "CSV", "summary": "Customer data"},
        ],
    }
)

print(result["messages"][-1].content)
```

### Tools

도구를 통해 모델이 데이터베이스, API 및 외부 시스템과 상호 작용할 수 있습니다.   
도구를 정의하고 선택하는 방법은 모델이 작업을 효과적으로 완료할 수 있는지에 직접적인 영향을 미칩니다.  

도구 정의

- 각 도구에는 명확한 이름, 설명, 인수 이름 및 인수 설명이 필요합니다.   
- 이것들은 단순한 메타데이터가 아니라 모델이 도구를 언제 어떻게 사용할지에 대한 추론을 안내합니다.

```python
@tool(parse_docstring=True)
def search_orders(user_id: str, status: str, limit: int = 10) -> str:
    """Search for user orders by status.

    Use this when the user asks about order history or wants to check
    order status. Always filter by the provided status.

    Args:
        user_id: Unique identifier for the user
        status: Order status: 'pending', 'shipped', or 'delivered'
        limit: Maximum number of results to return
    """
    return f"Found orders for {user_id} with status {status} (limit: {limit})"


agent = create_agent(model=model, tools=[search_orders])

result = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "Show me my pending orders for user_123"}
        ]
    }
)

print(result["messages"][-1].content)
```

State 기반 도구 선택

대화 단계에 따라 사용 가능한 도구를 동적으로 조정합니다.

```python
@tool
def public_search(query: str) -> str:
    """Public search - available to all users."""
    return f"Public results for: {query}"


@tool
def private_search(query: str) -> str:
    """Private search - requires authentication."""
    return f"Private results for: {query}"


@tool
def advanced_search(query: str) -> str:
    """Advanced search - requires authentication and conversation history."""
    return f"Advanced results for: {query}"
```

```python
@wrap_model_call
def state_based_tools(
    request: ModelRequest, handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """대화 State에 따라 도구 필터링"""
    state = request.state
    is_authenticated = state.get("authenticated", False)
    message_count = len(state["messages"])

    # 인증되지 않은 경우 공개 도구만 활성화
    if not is_authenticated:
        tools = [t for t in request.tools if t.name == "public_search"]
        request = request.override(tools=tools)
    elif message_count < 5:
        # 대화 초반에는 고급 도구 제한
        tools = [t for t in request.tools if t.name != "advanced_search"]
        request = request.override(tools=tools)

    return handler(request)
```

```python
# 인증되지 않은 사용자
result = agent.invoke(
    {
        "messages": [{"role": "user", "content": "Search for Python tutorials"}],
        "authenticated": False,
    }
)
print("Unauthenticated:", result["messages"][-1].content)

# 인증된 사용자
result = agent.invoke(
    {
        "messages": [{"role": "user", "content": "Search for Python tutorials"}],
        "authenticated": True,
    }
)
print("\nAuthenticated:", result["messages"][-1].content)
```

Runtime Context 기반 도구 선택   

사용자 권한에 따라 도구를 필터링합니다.

```python
@tool
def read_data(table: str) -> str:
    """테이블에서 데이터를 읽어옵니다."""
    return f"{table} 테이블에서 데이터를 읽었습니다."


@tool
def write_data(table: str) -> str:
    """테이블에 데이터를 작성합니다."""
    return f"{table} 테이블에 데이터를 작성했습니다."


@tool
def delete_data(table: str, data_id: str) -> str:
    """테이블에서 데이터를 삭제합니다."""
    return f"{table} 테이블에서 데이터를 삭제했습니다."


@dataclass
class UserRole:
    user_role: str
```

```python
@wrap_model_call
def context_based_tools(
    request: ModelRequest, handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """Runtime Context 권한에 따라 도구 필터링"""
    user_role = request.runtime.context.user_role

    if user_role == "admin":
        # 관리자는 모든 도구 사용 가능
        pass
    elif user_role == "editor":
        # 편집자는 삭제 도구를 사용할 수 없습니다.
        tools = [t for t in request.tools if t.name != "delete_data"]
        request = request.override(tools=tools)
    else:
        # 뷰어는 읽기 전용 도구만 사용할 수 있습니다.
        tools = [t for t in request.tools if t.name == "read_data"]
        request = request.override(tools=tools)

    return handler(request)


agent = create_agent(
    model=model,
    tools=[read_data, write_data, delete_data],
    middleware=[context_based_tools],
    context_schema=UserRole,
    system_prompt="사용자의 요구사항을 바로 수행해 주세요. 주어진 도구를 사용해 주세요. 사용할 도구가 없다면, 권한이 없다고 답변하세요.",
)
```

```python
# 뷰어
stream_graph(
    agent,
    inputs={"messages": [HumanMessage(content="User 테이블을 조회하세요.")]},
    context=UserRole(user_role="viewer"),
)

# 뷰어
stream_graph(
    agent,
    inputs={
        "messages": [HumanMessage(content="User 테이블에서 abc 레코드를 삭제해 주세요")]
    },
    context=UserRole(user_role="viewer"),
)
# 관리자
stream_graph(
    agent,
    inputs={
        "messages": [HumanMessage(content="User 테이블에서 abc 레코드를 삭제해 주세요")]
    },
    context=UserRole(user_role="admin"),
)
```

Model   

다양한 모델은 다양한 강점, 비용 및 컨텍스트 창을 가지고 있습니다. 작업에 적합한 모델을 선택하세요.

In [6]:
from langchain.chat_models import init_chat_model

# 모델을 미들웨어 외부에서 한 번만 초기화
large_model = init_chat_model("openai:gpt-4.1")
efficient_model = init_chat_model("openai:gpt-4.1-mini")

In [7]:
@wrap_model_call
def state_based_model(
    request: ModelRequest, handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """대화 길이에 따라 모델 선택"""
    message_count = len(request.messages)

    if message_count > 10:
        # 긴 대화 - 큰 컨텍스트 창을 가진 모델 사용
        model = large_model
        print(f"Using large model for {message_count} messages")
    else:
        # 짧은 대화 - 효율적인 모델 사용
        model = efficient_model
        print(f"Using efficient model for {message_count} messages")

    request = request.override(model=model) # request의 모델 설정을 선택된 모델로 교체, 다른 설정은 그대로 유지
    return handler(request) # 수정된 요청으로 호출

```python
agent = create_agent(
    model=efficient_model, tools=[search_tool], middleware=[state_based_model]
)

# 짧은 대화
result = agent.invoke({"messages": [{"role": "user", "content": "Hello"}]})
print(result["messages"][-1].content[:100])


```